In [ ]:
import pandas as pd
import json
import numpy as np
import re

In [ ]:
df_2020 = pd.read_excel('2020.xlsx', sheet_name=1)

In [ ]:
def get_row_with_column_names(path, sheetname=0):
    renglon = 0
    df = pd.read_excel(path,
                          sheet_name=sheetname,
                          skiprows=renglon,
                          nrows=10)
    while "CONCEPTO" not in df.columns:

        renglon += 1
        df = pd.read_excel(path,
                          sheet_name=sheetname,
                          skiprows=renglon,
                          nrows=10)
    return renglon

def reverse_dictio(dictio):
  reversed_entities = {}
  for key, value in dic_entities.items():
    reversed_entities[value] = key

  return reversed_entities

# aplicar diccionario de entidades
# filtro tres columnas
# aplicar función con apply
# reversed entities

# ffill concepto
# aplicar diccionario de sectores
# ffill number
# eliminar seccion
# rename columns

dic = {'A.  PRESUPUESTO DE FUNCIONAMIENTO': 'FUNCIONAMIENTO',
 'B.  PRESUPUESTO DE SERVICIO DE LA DEUDA PUBLICA': 'DEUDA',
'B.  PRESUPUESTO DE SERVICIO DE LA DEUDA PÚBLICA': 'DEUDA',

 'C.  PRESUPUESTO DE INVERSION': 'INVERSION'}

with open("dic_entities.json", 'r') as de:
  dic_entities = json.load(de)

with open("dic_sector.json", 'r') as de:
  dic_sector = json.load(de)

with open("dic_sec_ents.json", 'r') as de:
  dic_sec_ents = json.load(de)

with open("dic_tipo_gasto.json", 'r') as de:
  dic_tipo_gasto = json.load(de)

dic_cuentas = {0:'subproyecto',
               1:'proyecto',
               2:'subcuenta',
               3:'cuenta'}

In [ ]:
dic_sector_rev = reverse_dictio(dic_sector)

In [ ]:
renglon = get_row_with_column_names('2020.xlsx', sheetname=1)

df = pd.read_excel('2020.xlsx',
                     sheet_name=1, skiprows=renglon)
#df = df.iloc[1630: 2322]


for col in df.iloc[::, -3:].columns:
  df.loc[::, col] = df.loc[::, col].str.replace(',', '').astype(float)

df.columns = [s.strip() for s in df.columns]

df = df[~(df['CTA \nPROG'].isna()) | ~(df['CONCEPTO'].isna())]



rev_entities = reverse_dictio(dic_entities)

try:
  df.drop(columns=['Unnamed: 0', 'Unnamed: 1'], inplace=True)
except:
  print('Error')

df['CONCEPTO'] = df['CONCEPTO'].str.capitalize().str.strip()
df['CONCEPTO'] = df['CONCEPTO'].fillna(method='ffill')
df['Código de entidad'] = df['CONCEPTO'].map(rev_entities)
df['Tipo de gasto'] = df['CONCEPTO'].str.extract(pat="([ABC]{1}\..*)")[0]


Error


In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
for idx, row in df.iterrows():
  cuenta = row['CTA \nPROG']
  subcuenta = row['SUBC\nSUBP']
  proyecto = row['OBJG PROY']
  subproyecto = row['ORD \nSPRY']
  concepto = row['CONCEPTO']

  if not pd.isna(subcuenta) and pd.isna(proyecto) and not pd.isna(subproyecto):
    ls = concepto.split('/')
    a = ls[0]
    b = ls[1]
    df.loc[idx, 'OBJG PROY'] = ls[0]
    df.loc[idx, 'ORD  \nSPRY'] = ls[1]

In [ ]:
# diccionario de entidades

df2 = df.copy()

df2['seccion'] = df2['CONCEPTO'].str.extract(pat="(^Seccion:[\s]+\d{4})")[0]
df2['Código de entidad'] = df2['seccion'].str.extract(pat="(\d{4}$)")
df2['Entidad'] = None
for idx, row in df2.iterrows():
  try:
    if not pd.isna(row['Código de entidad']):
      df2.loc[idx, 'Entidad'] = df2.loc[idx + 1, 'CONCEPTO']
  except:
    print('error', idx)
info_ents = df2[['Código de entidad', 'Entidad']].dropna().reset_index(drop=True)
info_ents['Código de entidad'] = info_ents['Código de entidad'].astype(int).astype(str)
ents_2020 = dict(info_ents.set_index('Código de entidad').to_records())

with open('ents_2020.json', "w") as f:
  json.dump(ents_2020, f)

In [ ]:
df.columns

Index(['CTA \nPROG', 'SUBC\nSUBP', 'OBJG PROY', 'ORD \nSPRY', 'REC',
       'CONCEPTO', 'APORTE\n NACIONAL', 'RECURSOS \nPROPIOS', 'TOTAL',
       'Código de entidad', 'Tipo de gasto'],
      dtype='object')

In [ ]:
df['seccion'] = df['CONCEPTO'].str.extract(pat="(^Seccion:[\s]+\d{4})")[0]
df['seccion'].ffill(inplace=True)

df['Código de entidad'] = df['seccion'].str.extract(pat="(\d{4}$)").astype("Int64").astype(str)
df['Entidad'] = df['Código de entidad'].map(dic_entities)
df['Código de sector'] = df['Código de entidad'].map(dic_sec_ents).astype("Int64").astype(str)
df['Sector'] = df['Código de sector'].map(dic_sector)

dic_cuentas = {0:'subproyecto',
               1:'proyecto',
               2:'subcuenta',
               3:'cuenta'}

df['nans'] = df[['CTA \nPROG', 'SUBC\nSUBP', 'OBJG PROY', 'ORD \nSPRY']].isna().sum(axis=1)
df['Tipo de cuenta'] = df['nans'].map(dic_cuentas)

#df['Entidad'].fillna(method='ffill', inplace=True)
#df['Código de sector'].fillna(method='ffill', inplace=True)

df['Unidad'] = df['CONCEPTO'].str.extract(pat="(Unidad: \d{6})")[0]
df['Código de unidad'] = df['Unidad'].str.extract(pat="(\d{6})")[0]
print(df.columns)
df = df[df['REC'].isna()]

def funcion(row):
  if not pd.isna(row['Tipo de cuenta']):
    return row['CONCEPTO']
  else:
    return np.nan

df['Nombre de cuenta'] = df.apply(funcion, axis=1)

dic_unidades = {}
df.reset_index(drop=True, inplace=True)
for idx, row in df.iterrows():
  if not pd.isna(row['Unidad']):
    df.loc[idx, 'Unidad'] = df.loc[idx + 1, 'CONCEPTO']

df['Tipo de gasto'].ffill(inplace=True)
#df['Unidad'].ffill(inplace=True)
#df['Código de unidad'].ffill(inplace=True)

#df['TOTAL'] = df['TOTAL'].str.replace(',','_').astype(float)
df['Cuenta_n'] = df['CTA \nPROG']
df.drop(columns='REC', inplace=True)

dic_cuentas = df[df['Tipo de cuenta'] == 'cuenta'][['CTA \nPROG', 'CONCEPTO']].drop_duplicates().set_index('CTA \nPROG').to_dict()['CONCEPTO']
df['CTA \nPROG'] = df['CTA \nPROG'].map(dic_cuentas)
df['seleccion'] = None

Index(['CTA \nPROG', 'SUBC\nSUBP', 'OBJG PROY', 'ORD \nSPRY', 'REC',
       'CONCEPTO', 'APORTE\n NACIONAL', 'RECURSOS \nPROPIOS', 'TOTAL',
       'Código de entidad', 'Tipo de gasto', 'seccion', 'Entidad',
       'Código de sector', 'Sector', 'nans', 'Tipo de cuenta', 'Unidad',
       'Código de unidad'],
      dtype='object')


<ipython-input-158-7eb2aa437af6>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Nombre de cuenta'] = df.apply(funcion, axis=1)


In [ ]:
dic_unidades = dict(df[['Código de unidad', 'Unidad']].drop_duplicates().set_index('Código de unidad').to_records())

with open('units_2020.json', 'w') as f:
  json.dump(dic_unidades, f)

In [ ]:
df['Tipo de gasto'] = df['Tipo de gasto'].str.split(' ').str[1].str.capitalize()
df.loc[df['Tipo de gasto'] == 'Servicio', 'Tipo de gasto'] = 'Deuda'

In [ ]:
df['Subcuenta_n'] = df['SUBC\nSUBP']
df['Objeto_n'] = df['OBJG PROY']
df['Ordinal_n'] = df['ORD \nSPRY']

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
for value in df['CTA \nPROG'].dropna().unique():


  # slice por cuenta
  filtro = df[df['CTA \nPROG'] == value]
  dic_subcuenta = filtro[filtro['Tipo de cuenta'] == 'subcuenta'][['SUBC\nSUBP', 'CONCEPTO']].dropna().set_index('SUBC\nSUBP').to_dict()['CONCEPTO']
  df.loc[filtro.index, 'SUBC\nSUBP'] = filtro['SUBC\nSUBP'].map(dic_subcuenta)
  df.loc[filtro.index, 'seleccion'] = filtro['nans'].min()
  if value == 'Infraestructura de transporte férreo':
    print(dic_subcuenta)


  for value2 in filtro['SUBC\nSUBP'].dropna().unique():

    filtro2 = filtro[filtro['SUBC\nSUBP'] == value2]
    dic_proyecto = filtro2[filtro2['Tipo de cuenta'] == 'proyecto'][['OBJG PROY', 'CONCEPTO']].dropna().set_index('OBJG PROY').to_dict()['CONCEPTO']
    if value2 == 'Intersubsectorial transporte':
      print(dic_proyecto)

    try:
      df.loc[filtro2.index, 'OBJG PROY'] = filtro2['OBJG PROY'].map(dic_proyecto)
    except:
      print(value, value2)
      #print(filtro2['OBJG PROY'].map(dic_proyecto))




    df.loc[filtro2.index, 'seleccion'] = filtro2['nans'].min()
    for value3 in filtro2['OBJG PROY'].dropna().unique():

      filtro3 = filtro2[filtro2['OBJG PROY'] == value3]
      dic_subproyecto = filtro3[filtro3['Tipo de cuenta'] == 'subproyecto'][['ORD \nSPRY', 'CONCEPTO']].dropna().set_index('ORD \nSPRY').to_dict()['CONCEPTO']
      df.loc[filtro3.index, 'ORD \nSPRY'] = filtro3['ORD \nSPRY'].map(dic_subproyecto)
      df.loc[filtro3.index, 'seleccion'] = filtro3['nans'].min()

{600.0: 'Intersubsectorial transporte'}


In [ ]:
df = df[df['seleccion'] == df['nans']]

In [ ]:
df = (df
      .drop(columns=['seccion', 'nans', 'Tipo de cuenta', 'Nombre de cuenta', 'seleccion', 'CONCEPTO'])
      .rename(columns={'CTA  \nPROG':'Cuenta',
               'SUBC\nSUBP': 'Subcuenta',
               'OBJG PROY': 'Objeto',
               'ORD  \nSPRY': 'Ordinal',
               'APORTE\nNACIONAL': 'Aporte nacional',
               'RECURSOS \nPROPIOS': 'Recursos propios'}))

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
gastos = pd.read_csv('gastos.csv')
gastos = gastos[gastos['Año'] == 2020]

In [ ]:
# entidades que están en en dataset pero no en el diccionario de entidades actualizado
missing = set(df['Código de entidad'].unique()) - set(dic_entities.keys())

In [ ]:
# cambiar código de entidad al actualizado
df.loc[df['Sector'].isna(), 'Código de entidad'] = df.loc[df['Código de entidad'].isin(missing), 'Código de entidad'].map(ents_2020).map(rev_entities)

# nombre de entidad

df.loc[df['Sector'].isna(), 'Entidad'] = df.loc[df['Sector'].isna(), 'Código de entidad'].map(dic_entities)
# obtener código de sector

df.loc[df['Sector'].isna(), 'Código de sector'] = df.loc[df['Sector'].isna(), 'Código de entidad'].map(dic_sec_ents)
# obtener sector

df.loc[df['Sector'].isna(), 'Sector'] = df.loc[df['Sector'].isna(), 'Código de sector'].astype(str).map(dic_sector)


In [ ]:
df.to_csv('datos_desag_2020_cuentas.csv', index=False)

In [ ]:
nvls = df[['Cuenta', 'Subcuenta', 'Objeto', 'Ordinal', 'Cuenta_n', 'Subcuenta_n', 'Objeto_n', 'Ordinal_n']].drop_duplicates()

In [ ]:
dic_total = {}
for key, value in dic_cuentas.items():
  dic_subcuentas = {}
  filtro = nvls[nvls['Cuenta_n'] == key]
  lista_subcuentas = dict(filtro[['Subcuenta_n', 'Subcuenta']].dropna().set_index('Subcuenta_n').to_records())
  if len(lista_subcuentas.values()) > 0:
    print('Subcuentas', lista_subcuentas)
    for key_sub, subcuenta in lista_subcuentas.items():
      filtro_subcuentas = filtro[filtro['Subcuenta_n'] == key_sub]
      lista_objetos = dict(filtro_subcuentas[['Objeto_n', 'Objeto']].dropna().set_index('Objeto_n').to_records())
      if len(lista_objetos.values()) > 0:
        print('Objetos', lista_objetos)
        dic_objetos = {}
        for key_obj, objeto in lista_objetos.items():
          filtro_objetos = filtro_subcuentas[filtro_subcuentas['Objeto_n'] == key_obj]
          lista_ordinal = dict(filtro_objetos[['Ordinal_n', 'Ordinal']].dropna().set_index('Ordinal_n').to_records())
          if len(lista_ordinal.values()) > 0:
            print("Ordinal", lista_ordinal)
          dic_objetos[key_obj] = [objeto, lista_ordinal]
      dic_subcuentas[key_sub] = [subcuenta, dic_objetos]
  dic_total[key] = [value, dic_subcuentas]